In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

In [2]:
data = np.load("/content/drive/MyDrive/ml1/train.npy", allow_pickle=True)
data = data.item()
data_input = data['input']

In [3]:
labels = data['label']

In [ ]:
data_input

In [ ]:
labels.shape

(1080,)

In [4]:

X_train, X_test, y_train, y_test = train_test_split(data_input,labels , test_size=0.2, random_state=156)

In [5]:
X_train.shape

(864, 256, 64, 1)

# 전처리


In [6]:
labels = data['label']
labels.shape
repeated_arr = []
for i in range(len(labels)):
    repeated_arr.extend(np.full(64, labels[i]))

labels_1 = np.array(repeated_arr)
labels_1





array([1., 1., 1., ..., 0., 0., 0.])

In [ ]:
labels_1.shape

(69120,)

In [ ]:
y_test.mean()

0.5

In [ ]:
y_train.shape

(864,)

In [ ]:
X_train.shape

(864, 256, 64, 1)

In [11]:
X_train_1 = []
for i in range(864):
    data_input_m = X_train[i][:][:].squeeze().T
    X_train_1.append(data_input_m)

input_T = np.array(X_train_1)
input_T.shape

(864, 64, 256)

In [10]:
data_input_0 = input_T.reshape(1080*64,256)
df = pd.DataFrame(data_input_0)
data_input_0.shape

ValueError: cannot reshape array of size 14155776 into shape (69120,256)

In [12]:
X_test_1 = []
for i in range(216):
    data_input_m = X_test[i][:][:].squeeze().T
    X_test_1.append(data_input_m)

input_T_test = np.array(X_test_1)
input_T_test.shape

(216, 64, 256)

In [13]:
data_input_train = input_T.reshape(864*64,256)
df = pd.DataFrame(data_input_train)
data_input_train.shape

(55296, 256)

In [14]:
data_input_test = input_T_test.reshape(216*64,256)
df = pd.DataFrame(data_input_test)
data_input_test.shape

(13824, 256)

In [15]:
repeated_arr = []
for i in range(len(y_train)):
    repeated_arr.extend(np.full(64, y_train[i]))

labels_train = np.array(repeated_arr)
labels_train.shape




(55296,)

In [ ]:
labels_train.mean()

0.5

In [ ]:
repeated_arr_test = []
for i in range(len(y_test)):
    repeated_arr_test.extend(np.full(64, labels[i]))

labels_test = np.array(repeated_arr_test)
labels_test.shape

(13824,)

In [16]:
def fft(eeg_signal, sampling_rate):
    """
    주어진 EEG 신호에 대한 FFT를 계산하고 주파수 및 해당 주파수 성분의 파워를 시각화합니다.

    :param eeg_signal: EEG 신호 (1차원 numpy 배열)
    :param sampling_rate: 샘플링 주파수 (Hz)
    """
    n = len(eeg_signal)
    T = 1 / sampling_rate
    yf = np.fft.fft(eeg_signal)
    xf = np.fft.fftfreq(n, T)[:n//2]
    yf = 2.0/n * np.abs(yf[:n//2])
    return yf

128

In [131]:
FFT_train = []

for i in  range(864):
    for s in range(64):
        FFT_train.append(fft(input_T[i][s], 500))

In [132]:
FFT_test= []

for i in  range(216):
    for s in range(64):
        FFT_test.append(fft(input_T_test[i][s], 500))

In [133]:
FFT_test = np.array(FFT_test)
FFT_train = np.array(FFT_train)

In [134]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(FFT_train)
X_test_scaled = scaler.transform(FFT_test)

In [ ]:
FFT_test.shape

(13824, 128)

In [ ]:
FFT_train.shape

(55296, 128)

In [21]:
def plot_fft(eeg_signal, sampling_rate):
    """
    주어진 EEG 신호에 대한 FFT를 계산하고 주파수 및 해당 주파수 성분의 파워를 시각화합니다.

    :param eeg_signal: EEG 신호 (1차원 numpy 배열)
    :param sampling_rate: 샘플링 주파수 (Hz)
    """
    n = len(eeg_signal)
    T = 1 / sampling_rate
    yf = np.fft.fft(eeg_signal)
    xf = np.fft.fftfreq(n, T)[:n//2]
    yf = 2.0/n * np.abs(yf[:n//2])

    plt.plot(xf, yf)
    plt.title('FFT of EEG Signal')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Power')
    plt.grid()
    plt.show()
    return yf
# 예시 EEG 신호 (임의의 데이터)

In [ ]:
plot_fft(input_T[0][0], 256)

In [ ]:
data_input_0 = input_T.reshape(1080*64,256)
df = pd.DataFrame(data_input_0)
data_input_0.shape

(69120, 256)

In [ ]:
69120-66270

2850

K-fold val & xgb

In [135]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

In [ ]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

model = XGBClassifier()

# k-fold 교차 검증을 수행합니다.
scores = cross_val_score(model,FFT_train , labels_train, cv=kfold)

# 각 fold에서의 점수를 출력합니다.
for i, score in enumerate(scores):
    print(f"Fold {i+1} Score: {score}")

# 전체 fold의 평균 점수를 출력합니다.
print(f"Average Score: {scores.mean()}")

Fold 1 Score: 0.7556962025316456
Fold 2 Score: 0.7586581065195768
Fold 3 Score: 0.7547698706935527
Fold 4 Score: 0.7601953160321909
Fold 5 Score: 0.7563070802061669
Average Score: 0.7571253151966266


In [22]:
labels_train.mean()

0.5

In [136]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [137]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier



xgb_wrapper = XGBClassifier(n_estimators = 1000, learning_rate = 0.05, max_depth = 7)
xgb_wrapper.fit(X_train_scaled, labels_train)
w_preds = xgb_wrapper.predict(X_test_scaled)

----

In [ ]:
y_test.mean()

0.5

In [138]:
pred_label = list(w_preds)

In [139]:
print(pred_label)

[1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 

In [140]:
np.mean(pred_label)

0.5064380787037037

In [141]:
t = 0
while(t<=1):
    answer = []
    for i in range(216):
        pred = []
        for s in range(64):
            pred.append(pred_label[i*64+s])
        if sum(pred)/len(pred) > t:
            answer.append(1)
        else:
            answer.append(0)
    accuracy = accuracy_score(y_test, answer)
    print(accuracy,t)
    t+=0.01

0.5231481481481481 0
0.5231481481481481 0.01
0.5324074074074074 0.02
0.5324074074074074 0.03
0.5416666666666666 0.04
0.5509259259259259 0.05
0.5509259259259259 0.060000000000000005
0.5787037037037037 0.07
0.5925925925925926 0.08
0.5925925925925926 0.09
0.5972222222222222 0.09999999999999999
0.6111111111111112 0.10999999999999999
0.6111111111111112 0.11999999999999998
0.6111111111111112 0.12999999999999998
0.6111111111111112 0.13999999999999999
0.6203703703703703 0.15
0.6342592592592593 0.16
0.6342592592592593 0.17
0.6527777777777778 0.18000000000000002
0.6620370370370371 0.19000000000000003
0.6620370370370371 0.20000000000000004
0.6759259259259259 0.21000000000000005
0.6990740740740741 0.22000000000000006
0.6990740740740741 0.23000000000000007
0.7175925925925926 0.24000000000000007
0.7268518518518519 0.25000000000000006
0.7268518518518519 0.26000000000000006
0.7546296296296297 0.2700000000000001
0.7546296296296297 0.2800000000000001
0.7638888888888888 0.2900000000000001
0.7824074074074

#*TEST 파일 load 및 csv export*#


In [142]:
data_test = np.load("/content/drive/MyDrive/ml1/test.npy", allow_pickle=True)
data_test = data_test.item()
data_input_test = data_test['input']

In [143]:
input_test_T = []
for i in range(120):
    data_input_m = data_input_test[i][:][:].squeeze().T
    input_test_T.append(data_input_m)

input_test_T = np.array(input_test_T)
input_test_T.shape

(120, 64, 256)

In [144]:
FFT_test = []

for i in  range(120):
    for s in range(64):
        FFT_test.append(fft(input_test_T[i][s], 256))

FFT_test = np.array(FFT_test)
FFT_test.shape

(7680, 128)

In [145]:
X_test_scaled_test = scaler.transform(FFT_test)


In [146]:
y_pred = xgb_wrapper.predict(X_test_scaled_test)

In [147]:
y_pred.mean()

0.5723958333333333

In [148]:
csv = list(y_pred)

In [149]:
len(csv)

7680

In [162]:
answer = []

for i in range(120):
    pred = []
    for s in range(64):
        pred.append(csv[i*64+s])
    if sum(pred)/len(pred) > 0.62:
        answer.append(1)
    else:
        answer.append(0)

len(answer)

120

In [163]:
test_pred = np.array(answer)

In [164]:
test_pred.mean()

0.5

In [165]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [166]:
df = pd.DataFrame(answer)

In [167]:
df

,0
0,1
1,1
2,1
3,1
4,1
...,...
115,0
116,0
117,0
118,0


In [168]:
-df.mean()

0   -0.5
dtype: float64

In [169]:
df.to_csv('submission_37.csv',index = True)